In [2]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references('../../shared_data/training_mini/') 
# Importiere EKG-Dateien, zugehörige Diagnose, Sampling-Frequenz (Hz) und Name (meist fs=256 Hz)

100	 Dateien wurden geladen.


In [3]:
# -*- coding: utf-8 -*-

import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages
import mne
from scipy import signal as sig
import ruptures as rpt
import json
import pywt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.over_sampling import SMOTE

import numpy as np
from wettbewerb import load_references, get_3montages, get_6montages
import pywt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

number_montages = 3
N_samples = 2000 # Number of samples per division
# Decompose the wave
wavelet = 'db4'
scaler = StandardScaler()
new_signal = []

mont1_signal = []
mont2_signal = []
mont3_signal = []
whole_mont = [mont1_signal,mont2_signal,mont3_signal]
for i,_id in enumerate(ids):
    
    if number_montages == 6:
        _montage, _montage_data, _is_missing = get_6montages(channels[i], data[i])
    else:
        _montage, _montage_data, _is_missing = get_3montages(channels[i], data[i])
        
    _fs = sampling_frequencies[i]
    features_per_id = []

    for j, signal_name in enumerate(_montage):
        signal = _montage_data[j]
        # Notch-Filter to compensate net frequency of 50 Hz
        signal_notch = mne.filter.notch_filter(x=signal, Fs=_fs, freqs=np.array([50.,100.]), n_jobs=2, verbose=False)
        # Bandpassfilter between 0.5Hz and 70Hz to filter out noise
        signal_filter = mne.filter.filter_data(data=signal_notch, sfreq=_fs, l_freq=0.5, h_freq=70.0, n_jobs=2, verbose=False)
        # Defining number of divisions for signal
        N_div = len(signal_filter)//N_samples
        # Normalizing data
        norm_montage_data = scaler.fit_transform(signal_filter.reshape(-1,1)).reshape(1,-1)[0]
    
        for i in range(N_div):
            montage_array = norm_montage_data[i*N_samples:(i+1)*N_samples]
            whole_mont[j].append(montage_array)


labels = []
for i,_id in enumerate(ids):
    if eeg_labels[i][0]:
        onset = eeg_labels[i][1]
        offset = eeg_labels[i][2]
        sample_freq = sampling_frequencies[i]
        total_time = len(data[i][1])/sample_freq
        N_div = len(data[i][1])//N_samples
        for num in range(N_div):
            if (((total_time/N_div)*(num) <= onset) and ((total_time/N_div)*(num+1) > onset)) or (((total_time/N_div)*(num) >= onset) and ((total_time/N_div)*(num) < offset)):
                labels.append([1])
            else:
                labels.append([0])
    else:
        N_div = len(data[i][1])//N_samples
        for num in range(N_div):
            labels.append([0])
labels = np.reshape(labels, (1,-1))[0]

In [4]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
oversample = SMOTE()
undersample = RandomUnderSampler()

mont1_signal, labels1 = undersample.fit_resample(whole_mont[0],labels)
mont2_signal, labels2 = undersample.fit_resample(whole_mont[1],labels)
mont3_signal, labels3 = undersample.fit_resample(whole_mont[2],labels)

whole_mont_sampled = [mont1_signal,mont2_signal,mont3_signal]

In [7]:
import tensorflow
import keras
import tensorflow.keras.optimizers.Adam

ModuleNotFoundError: No module named 'tensorflow.keras.optimizers.Adam'

In [5]:
len(mont1_signal)

1412

In [6]:
np.unique(labels1, return_counts=True)

(array([0, 1]), array([706, 706]))

In [7]:
scales = range(1,128)
waveletname = 'morl'
train_size = len(labels1)

train_data_cwt = np.ndarray(shape=(train_size, 127, 127, 9))

for i in range(0,train_size):
    for j in range(len(whole_mont_sampled)):
        signal = whole_mont_sampled[j][i]
        coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:127]
        train_data_cwt[i, :, :, j] = coeff_

In [8]:
np.shape(train_data_cwt)

(1412, 127, 127, 9)

In [9]:
x_train = train_data_cwt[:1200,:,:,:]
x_test = train_data_cwt[1200:,:,:,:]
y_train = labels1[:1200]
y_test = labels1[1200:]

In [10]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()

In [11]:
img_x = 127
img_y = 127
img_z = 3
input_shape = (img_x, img_y, img_z)
 
#num_classes = 6
batch_size = 16
epochs = 10
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#y_train = keras.utils.np_utils.to_categorical(y_train)
#y_test = keras.utils.np_utils.to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

In [ ]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()
 
img_x = 127
img_y = 127
img_z = 9
input_shape = (img_x, img_y, img_z)
 
num_classes = 6
batch_size = 16
num_classes = 7
epochs = 10
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
 

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
 
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))